In [25]:
import pandas as pd
import numpy as np

import yfinance as yf


import requests
from datetime import datetime, timedelta, timezone


import warnings

warnings.filterwarnings('ignore')

MONTH_DIR = 'Dec 2024'


In [26]:
# Top 10 ASAs 

# 796425061	COOP
# 386195940	goETH
# 924268058	FRY
# 1284444444 ORA
# 2494786278 $monko
# 386192725	goBTC
# 1185173782 mALGO
# 287867876	OPUL
# 452399768	Vote
# 2200000000 TINY



In [27]:
# Stables
# 31566704 USDC
# 312769 USDT 
# 227855942 EURS
# 760037151 xUSD


In [28]:
def date_to_unix_timestamp(start_date_str, end_date_str):
  """
  Converts start and end dates in YYYY-MM-DD format to Unix timestamps.

  Args:
    start_date_str: Start date string in YYYY-MM-DD format.
    end_date_str: End date string in YYYY-MM-DD format.

  Returns:
    A tuple containing the start and end Unix timestamps.
  """

  start_date = datetime.strptime(start_date_str, "%Y-%m-%d").replace(tzinfo=timezone.utc)
  end_date = datetime.strptime(end_date_str, "%Y-%m-%d").replace(tzinfo=timezone.utc) + timedelta(days=1) - timedelta(seconds=1) 

  start_timestamp = int(start_date.timestamp())
  end_timestamp = int(end_date.timestamp())

  return start_timestamp, end_timestamp

In [29]:
def get_close_price(start_date, end_date, asset_id):
    """
    Fetches an asset historical data from the API for the given interval.
    
    Args:
        start_timestamp: start unix timestamp
        end_timestamp: end unix timestamp

    Returns:
        pd.DataFrame: Dataframe containing the fetched data in daily intervals.
    """
    start_unix, end_unix = date_to_unix_timestamp(start_date, end_date)

    price_feed = f'https://indexer.vestige.fi/assets/{asset_id}/candles?network_id=0&interval=86400&start={start_unix}&end={end_unix}&denominating_asset_id=0&volume_in_denominating_asset=false'

    response = requests.get(price_feed)
    data = response.json()
    df = pd.DataFrame(data)
    return df

In [30]:
start_date = '2024-12-01'
end_date = '2024-12-31'

stables_dfs = {}
stables_ids = {'usdc': 31566704, 'usdt': 312769,
              'euros': 227855942,'xusd':760037151}

for stablecoin, stables_id in stables_ids.items():
    stables_dfs[stablecoin] = get_close_price(start_date, end_date, stables_id)

In [31]:
start_date = '2024-12-01'
end_date = '2024-12-31'

asas_dfs = {}
asas_ids = {'coop': 796425061, 'goeth': 386195940, 'fry': 924268058, 'ora': 1284444444,
            'monko': 2494786278,  'gobtc': 386192725, 'malgo': 1185173782, 'opul': 287867876,
            'vote': 452399768, 'tiny': 2200000000}

for asa, asa_id in asas_ids.items():
    asas_dfs[asa] = get_close_price(start_date, end_date, asa_id)

In [32]:
def create_combined_df(dataframes):
  """
  Creates a single DataFrame with 'date' as the index and 
  'close' columns named after the stablecoin keys.

  Args:
    dataframes: A dictionary where keys are stablecoin names 
                and values are the corresponding DataFrames.

  Returns:
    A pandas DataFrame with the combined data.
  """

  combined_df = pd.DataFrame()

  for asset, df in dataframes.items():
    df['date'] = pd.to_datetime(df['timestamp'], unit='s') 
    df.set_index('date', inplace=True) 
    df.rename(columns={'close': asset}, inplace=True) 
    combined_df = combined_df.join(df[asset], how='outer') 

  return combined_df

# Example usage
stables_final_df = create_combined_df(stables_dfs) 
asas_final_df = create_combined_df(asas_dfs)

In [33]:
stables_final_df.to_csv(f'{MONTH_DIR}/Results/stablecoins_dec.csv')

In [34]:
asas_final_df.to_csv(f'{MONTH_DIR}/Results/asas_dec.csv')

In [35]:
algo_price = pd.read_csv(f'{MONTH_DIR}/algo_price.csv')
algo_price['snapped_at'] = pd.to_datetime(algo_price['snapped_at'])
price_feed = algo_price[(algo_price['snapped_at'] >= start_date) & (algo_price['snapped_at'] <= end_date)]
price_feed[['snapped_at', 'price']].to_csv(f'{MONTH_DIR}/Results/ALGO_dec.csv', index=False)

In [36]:
asa_volumes = pd.read_csv(f'{MONTH_DIR}/asa_volumes.csv')
asa_volumes['dt'] = pd.to_datetime(asa_volumes['dt'])


def multiply_columns(df1, df2):
  """
  Joins price and volume dataframes to convert into ALGO volumes

  Args:
    df1: Volumes Dataframe in native Currency
    df2: Asset/ALGO Price DataFrame.

  Returns:
    A merged DataFrame with ALGO Volumes
  """

  # Merge DataFrames on 'dt' and 'date'
  merged_df = df1.merge(df2, left_on='dt', right_on='date', suffixes=('_df1', '_df2'))

  # Calculate product of corresponding columns
  columns_to_multiply = ['coop', 'goeth', 'fry', 'ora', 'monko', 'gobtc', 'malgo', 'opul', 'vote', 'tiny']
  for col in columns_to_multiply:
      merged_df[col] = merged_df[col + '_df1'] * merged_df[col + '_df2']

  # Drop unnecessary columns
  merged_df.drop(columns=[col + '_df1' for col in columns_to_multiply] + 
                  [col + '_df2' for col in columns_to_multiply], 
             inplace=True)

  # Rename 'dt' column to 'date'
  merged_df.rename(columns={'dt': 'date'}, inplace=True)

  return merged_df


result_df = multiply_columns(asa_volumes, asas_final_df)

In [37]:
result_df.to_csv(f'{MONTH_DIR}/Results/asas_algo_volumes.csv', index=False)

In [38]:
# HAFN

def get_price_feed(ticker):
  """
  Fetches the price feed for the specified ticker for December 2024 
  using the yfinance library.

  Args:
    ticker: The ticker symbol (e.g., "AFNUSD=X").

  Returns:
    A pandas DataFrame containing the historical data for the specified period.
  """

  try:
    # Define start and end dates, consider forex does not operate 24/7
    start_date = "2024-11-29"
    end_date = "2025-01-01"

    # Download historical data using yfinance
    data = yf.download(ticker, start=start_date, end=end_date)
    return data
  except Exception as e:
    print(f"Error fetching data for {ticker}: {e}")
    return None

# Example usage
ticker = "AFNUSD=X"
december_data = get_price_feed(ticker)


[*********************100%%**********************]  1 of 1 completed


In [40]:
december_data.to_csv(f'{MONTH_DIR}/Results/HAFN_dec.csv')